### Run BASiCS for estimation method comparisons

In [1]:
library(Seurat)
library(SeuratData)
library(cowplot)
library(patchwork)
library(scHOT)
library(tidyverse)
library(dplyr)
library(BiocParallel)
library(readr)
library(Matrix)

Attaching SeuratObject

Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (is.na(desc)) {:
“the condition has length > 1 and only the first element 

In [2]:
library(Seurat)
library(SeuratData)
library(SeuratDisk)

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [3]:
library(BASiCS)

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQR

In [4]:
filenames = c('full', 'partial_1')
setwd('/data_volume/memento/method_comparison/basics/')


### Convert h5ad to seurat objects for simulations

In [5]:
setwd('/data_volume/memento/method_comparison/basics/')

# for (f in filenames){
    
#     filename = paste(
#         f,
#         '.h5ad',
#         sep=''
#     )
#     Convert(filename, dest = "h5seurat", overwrite = TRUE, verbose = FALSE)
# }

### BASiCS for simulations

In [16]:
runBASiCS <- function(seurat, stim){
    Idents(seurat) <- 'stim'
    dat <- subset(x = seurat, idents = stim)
    dat$batch <- sample(1:2, dim(dat)[2], replace=TRUE)
    dat.sce <- SingleCellExperiment(
    assays=list(counts = GetAssayData(dat)),
    colData = data.frame(BatchInfo = dat$batch))
    Chain <- BASiCS_MCMC(
      dat.sce,
      N = 20000, Thin = 20, Burn = 10000, WithSpikes = FALSE, SubsetBy = 'cell',
      PrintProgress = TRUE, Regression = TRUE,Threads = getOption("Ncpus", 90),)
    return(Chain)
}

In [ ]:
setwd('/data_volume/memento/method_comparison/basics/')

for (f in filenames){
    infilename = paste(
        f,
        '.h5seurat',
        sep=''
    )

    data_obj <- LoadH5Seurat(infilename, misc=F, verbose = FALSE)

#     ctrl_mu <- runBASiCS(data_obj, 'ctrl','mu')
#     stim_mu <- runBASiCS(data_obj, 'stim','mu')

    ctrl <- runBASiCS(data_obj, 'ctrl')
    stim <- runBASiCS(data_obj, 'stim')
    
    saveRDS(ctrl, file = paste(f, "_ctrl_20000.rds", sep=""))
    saveRDS(stim, file = paste(f, "_stim_20000.rds", sep=""))
}


### Run BASiCS DE/DV

In [118]:
seq(0,1,0.1)

[1] 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0

In [ ]:
for (f in filenames){
    
    ctrl <- readRDS(paste(f, "_ctrl_20000.rds", sep=""))
    stim <- readRDS(paste(f, "_stim_20000.rds", sep=""))
    
    for (fdr_thresh in seq(0,1,0.05)){
    
        Test <- BASiCS_TestDE(
          Chain1 = ctrl, Chain2 = stim,
          GroupLabel1 = "ctrl", GroupLabel2 = "stim",
          EpsilonM = 0, EpsilonD =0,
          EpsilonR = 0,
          EFDR_M = fdr_thresh, EFDR_D = fdr_thresh,EFDR_R=fdr_thresh, k=2,
          Offset = TRUE, PlotOffset = FALSE, Plot = FALSE
        )

        de <- as.data.frame(Test, Parameter = "Mean")
        dv <- as.data.frame(Test, Parameter = "ResDisp")
        
        write_csv(de, paste(f, "_ctrl_20000_de_", fdr_thresh,'.csv',sep=""))
        write_csv(dv, paste(f, "_ctrl_20000_dv_", fdr_thresh,'.csv',sep=""))
    }
}